## Prepare data

In [1]:
import pandas as pd
import numpy as np
import sklearn as sk

In [2]:
train = pd.read_csv('train_small.csv')

/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (7,8) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
train.dropna(subset=['type'], inplace=True)

In [4]:
test = pd.read_csv('labeled_output.csv')

In [5]:
test.dropna(subset=['text'], inplace=True)

In [6]:
test['category']=np.nan

In [7]:
X = train['text']
y = train['category']

In [8]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(X, y, random_state=32, test_size=0.2)

In [9]:
from sklearn.preprocessing import LabelEncoder

encoder = LabelEncoder()
y_train = encoder.fit_transform(y_train)
y_test = encoder.fit_transform(y_test)

In [10]:
# from sklearn.preprocessing import StandardScaler
# scaler = StandardScaler()

# scaler.fit(x_train)  # Don't cheat - fit only on training data
# x_train = scaler.transform(x_train)
# x_test = scaler.transform(x_test) 

## SVM classifier

In [17]:
from sklearn.feature_extraction.text import TfidfVectorizer

tf_vec = TfidfVectorizer(analyzer='word',
                         min_df=1,
                         strip_accents='unicode',
                         token_pattern=r'\w{1,}',
                         ngram_range=(2,3),
                         )
x_train_vec = tf_vec.fit_transform(x_train)
x_test_vec = tf_vec.transform(x_test)

In [18]:
from sklearn.linear_model import SGDClassifier as SGD

svm = SGD(shuffle=True,
          loss='hinge', 
          penalty='l2',
          alpha=1e-3, 
          n_iter=5, 
          random_state=42,
          )     

In [19]:
svm.fit(x_train_vec, y_train)

/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/stochastic_gradient.py:117: DeprecationWarning: n_iter parameter is deprecated in 0.19 and will be removed in 0.21. Use max_iter and tol instead.
  DeprecationWarning)


SGDClassifier(alpha=0.001, average=False, class_weight=None, epsilon=0.1,
       eta0=0.0, fit_intercept=True, l1_ratio=0.15,
       learning_rate='optimal', loss='hinge', max_iter=None, n_iter=5,
       n_jobs=1, penalty='l2', power_t=0.5, random_state=42, shuffle=True,
       tol=None, verbose=0, warm_start=False)

In [20]:
svm_pred = svm.predict(x_test_vec)

In [21]:
from sklearn.metrics import accuracy_score
print(accuracy_score(svm_pred, y_test))

0.5769230769230769


In [ ]:
from sklearn.model_selection import GridSearchCV

parameters = {loss: ['hinge','log'], 'tfidf__use_idf': (True, False), 'clf__alpha': (1e-2, 1e-3)}

In [ ]:
grid = {
    'alpha': [1e-4, 1e-3, 1e-2, 1e-1, 1e0, 1e1, 1e2, 1e3], # learning rate
    'n_iter': [1000], # number of epochs
    'loss': ['log','hinge'], # logistic regression,
    'penalty': ['l2'],
    '
}
paramGrid = ParameterGrid(grid)

bestModel, bestScore, allModels, allScores = 
pf.bestFit(SGDClassifier, paramGrid,
           X_train, y_train, X_val, y_val, 
           metric = roc_auc_score, bestScore='max',
           scoreLabel = "AUC")

print(bestModel, bestScore)

In [ ]:
parameters = {'kernel':('linear', 'rbf'), 'C':(1,0.25,0.5,0.75),'gamma': (1,2,3,'auto'),'decision_function_shape':('ovo','ovr'),'shrinking':(True,False)}
clf = GridSearchCV(svm, parameters)
clf.fit(X_train,Y)